In [1]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os

# 源Excel文件路径
source_file_path = '../forhx/对账明细查询表_20240701_17时06分.xlsx'
# 目标Excel文件路径
target_file_path = '../forhx/对账单2024.06月.xlsx'

# 使用pandas读取Excel文件的元数据，但不加载数据
xls = pd.ExcelFile(target_file_path,engine='openpyxl')

# 获取所有工作表的名称
sheet_names = xls.sheet_names

sourcefile=pd.read_excel(source_file_path,sheet_name=0) #会自动关闭文件
print('====================================info=======================================')
print(sourcefile.info())
print('===============================================================================')
        
        
def new_columnOrder(supplynamevalue,df):
    new_order = ['变动时间', '变动说明', '银行','常规回款','现金核销回款','非现金核销回款','现金核销订单金额','非现金核销订单金额','常规订单金额','调换货回款','换货订单金额']  # 新的列顺序
    df = df[new_order].reset_index(drop=True)  # 重新排序列 df.reset_index(drop=True)
    
   
    # 特定列插入固定值
    df.insert(loc=2,column='保证金',value='')
    df.insert(loc=3,column='工行',value='') 
    df.insert(loc=4,column='招行',value='')
    df.insert(loc=5,column='订单结案-现金',value='')
    df.insert(loc=6,column='订单结案-现金核销',value='')
    df.insert(loc=7,column='订单结案-非现金核销',value='')
    df.insert(loc=8,column='核销-现金核销',value='')
    df.insert(loc=9,column='核销-非现金核销',value='')
    df.insert(loc=10,column='出库-现金核销',value='') 
    df.insert(loc=11,column='出库-非现金核销',value='')
    df.insert(loc=12,column='出库-现金',value='')
    df.insert(loc=13,column='出库-调换货',value='')
    
    for lineindex in range(df.shape[0]):
        if df.loc[lineindex,'银行']=='工行':
            df.loc[lineindex,'工行']=df.loc[lineindex, '常规回款']
            df.loc[lineindex,'招行']=''
        elif df.loc[lineindex,'银行']=='招行':
            df.loc[lineindex,'工行']=''
            df.loc[lineindex,'招行']=df.loc[lineindex, '常规回款']   
        else:
            print(supplynamevalue+" 既不是招行，也不是工行，将会赋值为空")
            df.loc[lineindex,'工行']=''
            df.loc[lineindex,'招行']=''
    for lineindex in range(df.shape[0]):
        if df.loc[lineindex,'变动说明']=='回款方式其他':
            df.loc[lineindex,'订单结案-现金']=df.loc[lineindex,'常规回款']
        elif df.loc[lineindex,'变动说明']=='回款方式核销回款-现金核销':
            df.loc[lineindex,'核销-现金核销']=df.loc[lineindex,'现金核销回款']
           
        elif df.loc[lineindex,'变动说明']=='回款方式核销回款-非现金核销': 
            df.loc[lineindex,'核销-非现金核销']=df.loc[lineindex,'非现金核销回款']
        elif df.loc[lineindex,'变动说明']=='核销订单':
          
            df.loc[lineindex,'出库-现金核销']=df.loc[lineindex,'现金核销订单金额']
            df.loc[lineindex,'出库-非现金核销']=df.loc[lineindex,'非现金核销订单金额']
        elif  df.loc[lineindex,'变动说明']=='正常订单-账余常规':
            df.loc[lineindex,'出库-现金']=df.loc[lineindex,'常规订单金额']
        elif df.loc[lineindex,'变动说明']=='退货单': 
            df.loc[lineindex,'出库-调换货']=df.loc[lineindex,'调换货回款']
        elif df.loc[lineindex,'变动说明']=='换货订单-账余常规':
            df.loc[lineindex,'出库-调换货']=df.loc[lineindex,'换货订单金额']
        else:
            print('变动说明列='+df.loc[lineindex,'变动说明']+' 不在预先设定的值（回款方式其他,回款方式核销回款-现金核销,正常订单-账余常规...）跳过不处理')
            
            
            
            
    # 定义一个列名列表，列出你想要删除的列
    columns_to_delete = ['银行','常规回款','现金核销回款','非现金核销回款','现金核销订单金额','非现金核销订单金额','常规订单金额','调换货回款','换货订单金额']

    # 循环遍历列名列表，逐一删除列
    for column in columns_to_delete:
        if column in df.columns:
            del df[column]
        else:
            print(f"Column {column} not found in DataFrame.")
            break
    return df      
    
def handexcelSheet(sheetname):
    # 通过工作表名称读取
    subsheetinfo = xls.parse(sheetname)

    try:
        supplynamekey = subsheetinfo.iloc[0, 0]
        supplynamevalue = subsheetinfo.iloc[0, 1]
        print(supplynamekey,supplynamevalue)
    except KeyError:
        print('the sheet:'+sheetname+'is not my target sheet, please next.')
        return
        
    if supplynamekey=='供应商名称：':
        
        # 选取代理商名称列是supplynamevalue的行
        filtered_rows = sourcefile[sourcefile['代理商名称'] == supplynamevalue]
        print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        print(filtered_rows)
        print(filtered_rows.shape[0])
        print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        if filtered_rows.shape[0]==0:
            print(source_file_path+' 文件中没有找到 '+supplynamevalue+' 对应的信息')
        else:
            df_order=new_columnOrder(supplynamevalue,filtered_rows)
            print("=========================将要拷贝到目标文件的内容========================\n")
            print(df_order)
            print('===========================================================================')
            print('写文件前大小为 '+str(os.path.getsize(target_file_path)))
            # 追加df_order到Excel文件的特定工作表中
            with pd.ExcelWriter(target_file_path, engine='openpyxl', mode='a') as writer:  # 使用'a'模式追加写入   
                df_order.to_excel(writer, sheet_name=sheetname, index=False, header=False)  # 设置header=False避免重复写入列名
            print('写文件后大小为 '+str(os.path.getsize(target_file_path)))
    else:
        print ('read next sheet') 
    
    
#handexcelSheet('河北瑞俪美')


count=0
# 打印所有工作表名称
for name in sheet_names:
    count+=1
    print('一共有'+str(len(sheet_names))+'个工作表，正在处理第'+str(count)+'个工作表->'+name)
    handexcelSheet(name)
    if count==10:
        break




====================================info=======================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 15 columns):
Unnamed: 0    26 non-null object
代理商名称         1285 non-null object
变动时间          1285 non-null datetime64[ns]
变动说明          1285 non-null object
关联回款          1285 non-null object
银行            100 non-null object
常规回款          1285 non-null float64
非现金核销回款       1259 non-null float64
现金核销回款        1259 non-null float64
调换货回款         112 non-null float64
关联订单          1259 non-null object
常规订单金额        1259 non-null float64
非现金核销订单金额     1259 non-null float64
现金核销订单金额      1259 non-null float64
换货订单金额        1259 non-null float64
dtypes: datetime64[ns](1), float64(8), object(6)
memory usage: 150.7+ KB
None
一共有262个工作表，正在处理第1个工作表->本月期初（原）
序号 代理商
read next sheet
一共有262个工作表，正在处理第2个工作表->本月期初
品牌 序号
read next sheet
一共有262个工作表，正在处理第3个工作表->本月发生（护肤）
品牌 序号
read next sheet
一共有262个工作表，正在处理第4个工作表->本月发生（洗护）
品牌 序号
re

D:\ProgramFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:121: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



=========================将要拷贝到目标文件的内容========================

                                     拉萨天芮商贸有限公司-往来对账单       Unnamed: 1  \
0                                              供应商名称：  河北瑞俪美企业管理咨询有限公司   
1                                                  日期               摘要   
2                                                 NaN              NaN   
3                                                 NaN              NaN   
4                                 2024-04-30 00:00:00               合计   
5                                 2024-05-06 10:50:00  正常订单-老店季度政策款-常规   
6                                 2024-05-31 14:41:00    回款方式核销回款-现金核销   
7                                 2024-05-31 14:41:00   回款方式核销回款-非现金核销   
8                                                 NaN              NaN   
9                                                 NaN              NaN   
10                                                NaN              NaN   
11                                               

写文件后大小为 2023060
一共有262个工作表，正在处理第8个工作表->成都盛泽西
供应商名称： 成都市盛泽西企业管理咨询有限公司
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Empty DataFrame
Columns: [Unnamed: 0, 代理商名称, 变动时间, 变动说明, 关联回款, 银行, 常规回款, 非现金核销回款, 现金核销回款, 调换货回款, 关联订单, 常规订单金额, 非现金核销订单金额, 现金核销订单金额, 换货订单金额]
Index: []
0
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
../forhx/对账明细查询表_20240701_17时06分.xlsx 文件中没有找到 成都市盛泽西企业管理咨询有限公司 对应的信息
一共有262个工作表，正在处理第9个工作表->大连美慧


error: Error -3 while decompressing data: invalid distance too far back